In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

Using TensorFlow backend.


In [2]:
BASE_DIR='/Users/magictavern/Downloads'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove')
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [3]:
# first, build index mapping words in the embeddings set
# to their embedding vector
print('Indexing word vectors.')
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding = 'utf-8') as f:#use pretrained word_vector
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [4]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}#utf-8 for Chinese
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header (return the position of '\n\n')
                    if 0 < i: #no '\n\n'
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)
print('Found %s texts.' % len(texts))

Found 19997 texts.


In [5]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 174071 unique tokens.


In [6]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)#padding
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [7]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])#20% validation

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [20]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing embedding matrix.


In [21]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embedding
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [22]:
print('Training model.')
# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Training model.
Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 107s 7ms/step - loss: 2.4444 - acc: 0.1948 - val_loss: 1.9005 - val_acc: 0.3378
Epoch 2/10
15998/15998 [==============================] - 102s 6ms/step - loss: 1.5656 - acc: 0.4592 - val_loss: 1.3256 - val_acc: 0.5414
Epoch 3/10
15998/15998 [==============================] - 103s 6ms/step - loss: 1.1965 - acc: 0.5917 - val_loss: 1.0870 - val_acc: 0.6199
Epoch 4/10
15998/15998 [==============================] - 105s 7ms/step - loss: 0.9810 - acc: 0.6692 - val_loss: 1.0851 - val_acc: 0.6367
Epoch 5/10
15998/15998 [==============================] - 107s 7ms/step - loss: 0.8309 - acc: 0.7185 - val_loss: 1.0500 - val_acc: 0.6562
Epoch 6/10
15998/15998 [==============================] - 101s 6ms/step - loss: 0.7041 - acc: 0.7652 - val_loss: 1.1214 - val_acc: 0.6522
Epoch 7/10
15998/15998 [==============================] - 102s 6ms/step - loss: 0.6047 - acc: 0.7973 - val_